In [1]:
########################################################################
#######   Elliptic Curve Operations over a finite field F_p     ########
########################################################################
################## INPUT PARAMETERS ######################

#p = int(input("Enter prime p: ").strip())           # e.g., 5
#curve_a = int(input("Enter coefficient a: ").strip())     # e.g., 1
#curve_b = int(input("Enter coefficient b: ").strip())     # e.g., 1
#Px = int(input("Enter generator x: ").strip())      # e.g., 0
#Py = int(input("Enter generator y: ").strip())      # e.g., 1
#P = (Px, Py)

p = 5
curve_a = 1
curve_b = 2
P = (1, 2)  # Generator point

########################################################################
############################     UTILS     #############################
########################################################################
def is_on_curve(P):
    """Return True if point P lies on the curve y^2 = x^3 + ax + b (mod p)."""
    #None means infinity
    if P is None:
        return True
    x, y = P
    return (y*y - (x*x*x + curve_a*x + curve_b)) % p == 0
    

########################################################################
###############        Multiplicative Inverse         ##################
########################################################################
def inv_mod(k, p):
    """Modular inverse of k mod p."""
    k = k % p
    if k == 0:
        raise ZeroDivisionError("No modular inverse for 0 (mod p)")
    return pow(k, -1, p)

########################################################################
############################ EC ARITHMETIC #############################
########################################################################
def ec_add(P, Q):
    """Add two EC points P and Q. Points are tuples (x,y) or None for infinity."""
    if P is None:
        return Q
    if Q is None:
        return P

    x1, y1 = P
    x2, y2 = Q

    # P + Q = infinity 
    if x1 == x2 and (y1 + y2) % p == 0:
        return None

    if P == Q:
        # Point doubling
        if y1 % p == 0:
            return None
        m = ((3 * x1 * x1 + curve_a) * inv_mod(2 * y1, p)) % p
    else:
        # Point addition
        if x1 == x2:
            return None
        m = ((y2 - y1) * inv_mod(x2 - x1, p)) % p

    x3 = (m * m - x1 - x2) % p
    y3 = (m * (x1 - x3) - y1) % p
    return (x3, y3)

########################################################################
############################ Scalar Multiplication #####################
########################################################################
def ec_mul(k, P):
    """Scalar multiplication k * P using double-and-add."""
    if not is_on_curve(P):
        raise ValueError(f"ec_mul: input point {P} is not on the curve")
    R = None
    Q = P
    while k:
        if k & 1:
            R = ec_add(R, Q)
        Q = ec_add(Q, Q)
        k >>= 1
    return R

########################################################################
#################### Finding order of an element  ######################
########################################################################
def ec_ord(P):
    """Return the order of point P."""
    if not is_on_curve(P):
        raise ValueError(f"Point {P} is not on the curve")
    for k in range(1, 2*p + 1):
        if ec_mul(k, P) is None:
            return k
    return None


In [2]:
########################################################################
######################### MAIN DEMO ####################################
########################################################################
print(f"Curve: y^2 = x^3 + {curve_a}x + {curve_b} (mod {p})")
print(f"The following are the non-infinity points on the elliptic curve over F_{p}:")
for x in range(0, p):
    for y in range (0, p):
        if (y*y - (x*x*x + curve_a*x + curve_b)) % p == 0:
            print(f"({x}, {y})")

print(f"\nGenerator P = {P}, on curve? {is_on_curve(P)}")

# Compute order of P
order_P = ec_ord(P)
print(f"\nOrder of P = {order_P}")

# Compute multiples of P
print("\nMultiples of P:")
for k in range(1, order_P+1):
    print(f"{k}*P = {ec_mul(k, P)}")

    
# Alice ephemeral key
import random as ran
alice_priv = ran.randint(1, order_P-1)
print(f"\nAlice's private key = {alice_priv}")
A = ec_mul(alice_priv, P)
print(f"Alice's public key A = {A}")

# Bob ephemeral key
bob_priv = ran.randint(1, order_P-1)
print(f"\nBob's private key = {bob_priv}")
B = ec_mul(bob_priv, P)
print(f"Bob's public key B = {B}")

# Shared secret
S_alice = ec_mul(alice_priv, B)
S_bob   = ec_mul(bob_priv, A)
print(f"\nShared secret computed by Alice: {S_alice}")
print(f"Shared secret computed by Bob:   {S_bob}")
assert S_alice == S_bob, "Shared secrets do not match!"
print("Shared secret matches! DH key exchange successful.")

Curve: y^2 = x^3 + 1x + 2 (mod 5)
The following are the non-infinity points on the elliptic curve over F_5:
(1, 2)
(1, 3)
(4, 0)

Generator P = (1, 2), on curve? True

Order of P = 4

Multiples of P:
1*P = (1, 2)
2*P = (4, 0)
3*P = (1, 3)
4*P = None

Alice's private key = 1
Alice's public key A = (1, 2)

Bob's private key = 2
Bob's public key B = (4, 0)

Shared secret computed by Alice: (4, 0)
Shared secret computed by Bob:   (4, 0)
Shared secret matches! DH key exchange successful.
